# Summarizing Feeds and Subscriptions
This notebook demonstrates how to describe available [Analytics Feeds](https://developers.planet.com/docs/analytics/#analytic-feeds) and [Subscriptions](https://developers.planet.com/docs/analytics/#subscriptions) with the Planet Analytic API.

## Setup
To use this notebook, you need to have the following:
- A Planet account with access to the Analytics API
- A Planet API Key

#### Set API Key

In [1]:
import os

# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', '0d687fe61f24476d8eac72d9f2034a7f')
# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

#### Set the base url for the Planet Analytic Feeds product
See the [Analytics API Docs](https://developers.planet.com/docs/analytics/) for more details.

In [2]:
BASE_URL = "https://api.planet.com/analytics/"

#### Test API Connection

In [3]:
import requests

feed_list_url = f'{BASE_URL}feeds?limit=1000'
resp = requests.get(feed_list_url, auth=BASIC_AUTH)
if resp.status_code == 200:
    print('Yay, you can access the Analytics API')
else:
    print('Something is wrong:', resp.content)

Yay, you can access the Analytics API


## Summarizing Feeds
In this section, we will see describe the available feeds.

#### How many feeds are there?

In [4]:
limit = 1000
feed_list_url = f'{BASE_URL}feeds?limit={limit}'
print(f'Feeds endpoint: {feed_list_url}')
resp = requests.get(feed_list_url, auth=BASIC_AUTH)
feeds = resp.json()['data']
feed_count = len(feeds)
print(f'Available feeds: {feed_count}')
if feed_count >= limit:
    print('More feeds are probably available through pagination links')
    print(resp.json()['links'])

Feeds endpoint: https://api.planet.com/analytics/feeds?limit=1000
Available feeds: 19


#### Inspecting feed metadata

In [5]:
from pprint import pprint
pprint(feeds[0])

{'created': '2020-02-15T02:06:30.159Z',
 'description': 'Building construction, Global Monthly, no UDM: Change v1.3',
 'id': '899de0a6-55de-4c2d-8f4e-215213553380',
 'interval': 120,
 'links': [{'href': 'https://api.planet.com/analytics/feeds/899de0a6-55de-4c2d-8f4e-215213553380',
            'rel': 'self',
            'type': 'application/json'},
           {'href': 'https://api.planet.com/analytics/feeds',
            'rel': 'feeds',
            'type': 'application/json'}],
 'process': {'actions': [{'Fn::SetVar': ['segment_task_list',
                                         {'Fn::Map': ['quad_prn',
                                                      ['{{ '
                                                       '.Source.global_monthly '
                                                       '| '
                                                       'RenderSourceResources '
                                                       '}}'],
                                              

**Some of the fields include:**
- id: this is a unique identifier for a feed
- title: a human friendly name for the feed
- description: more detail text about the feed
- created: timestamp for when the feed was originally created
- updated: timestamp for when the feed was last modified
- source: a blob describing the imagery on which the feed is based
- target: a blob describing the feed's output format
- links: a list of blobs containing urls to related resources

#### Showing all available feeds in a table
We can use a pandas DataFrame to summarize the available feeds.

In [6]:
import pandas as pd
# bump this up in case there are many available feeds to display
pd.options.display.max_rows = 1000
# make a dataframe from the feeds json data
df = pd.DataFrame(feeds)
# instead of including the entire source and target dicts, make columns for the types
df['targetType'] = df['target'].map(lambda t: t['type'])
df['sourceType'] = df['source'].map(lambda t: t[0]['type'])
df[['id', 'title', 'description', 'sourceType', 'targetType', 'created', 'updated']]


,id,title,description,sourceType,targetType,created,updated
0,899de0a6-55de-4c2d-8f4e-215213553380,"Building construction, Global Monthly, no UDM:...","Building construction, Global Monthly, no UDM:...",mosaic,collection,2020-02-15T02:06:30.159Z,2020-02-15T02:06:30.159Z
1,ec61ef60-d25b-407f-b678-f1a623893acd,"Building construction, Global Monthly, no UDM:...","Building construction, Global Monthly, no UDM:...",mosaic,mosaic,2020-02-15T02:02:34.820Z,2020-02-15T02:02:34.820Z
2,02fb385b-ca14-4ec8-9c51-e3702eaa7a63,"Building construction, Global Monthly, UDM: Ch...","Building construction, Global Monthly, UDM: Ch...",mosaic,collection,2020-02-11T23:27:53.263Z,2020-02-11T23:27:53.263Z
3,51160ccf-bbcb-467a-a091-b3862db468d5,"Building construction, Global Monthly, UDM: Se...","Building construction, Global Monthly, UDM: Se...",mosaic,mosaic,2020-02-11T23:22:41.851Z,2020-02-11T23:22:41.851Z
4,c1f878bc-e84d-4d40-ab86-795bafef0363,PROSERVE-ONLY - Northern Sea Route Ship Detec...,PROSERVE-ONLY - NSR Ships - Full Browse Product,image,collection,2019-12-19T01:30:31.130Z,2019-12-19T17:10:01.244Z
5,5c3e9935-f974-4920-9d0f-b0a065829909,PROSERVE-ONLY - Northern Sea Route Ship Detec...,PROSERVE-ONLY - NSR Ships - Tiles,image,collection,2019-12-19T01:29:39.914Z,2019-12-19T17:06:40.430Z
6,349d87e0-aed7-4ce8-8ceb-560a62e426a3,Roads (v7 Quarterly),Quarterly Road Detection (v7),mosaic,mosaic,2019-11-14T21:32:42.577Z,2019-11-14T21:32:42.577Z
7,198fd789-8019-485a-9863-4b8b1a4ff2c5,Buildings (v7 Quarterly),Quarterly Building Detection (v7),mosaic,mosaic,2019-11-14T21:31:59.073Z,2019-11-14T21:31:59.073Z
8,a30a7f6a-c526-40d4-b8a6-5d62832958c7,Ship Detections - No Ground Lock,Ship detections from rectified and non-rectifi...,image,collection,2019-10-24T14:54:11.853Z,2019-10-24T14:54:11.853Z
9,f5054991-f425-442a-b834-8abe054c46f8,Monthly Silo Bag Detections,Silo bag detections from global monthly mosaics.,mosaic,collection,2019-06-18T20:54:56.329Z,2019-06-18T20:54:56.329Z


## Summarizing Subscriptions
Now that we know about available feeds, let's check out available subscriptions.

In [7]:
limit = 1000
subscriptions_url = f'{BASE_URL}subscriptions?limit={limit}'
print(f'Subscriptions endpoint: {subscriptions_url}')
resp = requests.get(subscriptions_url, auth=BASIC_AUTH)
subs = resp.json()['data']
sub_count = len(subs)
print(f'Available subscriptions: {sub_count}')
if sub_count >= limit:
    print('More subscriptions are probably available through pagination links')
    print(resp.json()['links'])

Subscriptions endpoint: https://api.planet.com/analytics/subscriptions?limit=1000
Available subscriptions: 429


#### What's in a subscription?

In [8]:
pprint(subs[0])

{'created': '2020-02-19T16:00:16.831Z',
 'description': 'NRO AOIs: Cairo, Aleppo, Engels AF',
 'endTime': '2020-02-08T00:00:00.000Z',
 'feedID': '899de0a6-55de-4c2d-8f4e-215213553380',
 'geometry': {'coordinates': [[[[46.2865483435179, 51.5238803899017],
                                [46.1424372793634, 51.5253724085522],
                                [46.1401931105756, 51.435472685135],
                                [46.2840212665101, 51.4339854325255],
                                [46.2865483435179, 51.5238803899017]]],
                              [[[31.3083874363058, 30.1230990949877],
                                [31.3083874363058, 29.7036857092783],
                                [31.8024105729672, 29.6972905877681],
                                [31.7992130122121, 30.1284283629128],
                                [31.3083874363058, 30.1230990949877]]],
                              [[[37.2421471540336, 36.2689431949095],
                                [37.130871

Subscriptions also have id, title, description, created, and updated fields.
Additionally, there are fields for:
- feedID: which feed this subscription is for
- startTime: timestamp for the subscription's beginning
- endTime: timestamp for the subscription's ending
- geometry: spatial area of interest to which the subscription has access

**Important:** 
Subscriptions will only get results for source imagery observed between the `startTime` and `endTime` within the specified `geometry`.

`created` and `updated` refer to when the subscription itself was set up or modified and **do not** impact results that show up for the subscription. 

`startTime` and `endTime` **do** limit feed results for the subscription.

#### Showing all available subscriptions

In [9]:
df = pd.DataFrame(subs)
df[['id', 'title', 'description', 'feedID', 'startTime', 'endTime', 'created', 'updated']]

,id,title,description,feedID,startTime,endTime,created,updated
0,69ae8866-9c25-4f98-9815-a35fed904959,"Building construction Change, no UDM, AN-3408","NRO AOIs: Cairo, Aleppo, Engels AF",899de0a6-55de-4c2d-8f4e-215213553380,2018-02-01T00:00:00.000Z,2020-02-08T00:00:00.000Z,2020-02-19T16:00:16.831Z,2020-02-20T09:50:38.827Z
1,179ee514-85ec-4037-b7c1-0bdba8ea769f,"Building construction Seed state, no UDM, AN-3408","NRO AOIs: Cairo, Aleppo, Engels AF",ec61ef60-d25b-407f-b678-f1a623893acd,2018-01-01T00:00:00.000Z,2018-02-01T00:00:00.000Z,2020-02-19T01:49:16.267Z,2020-02-20T09:50:40.055Z
2,9e8e97ba-ff1e-42cf-a369-3a3bde05a7ee,"Building construction Change, no UDM, AN-3458",Diverse geographies around the world,899de0a6-55de-4c2d-8f4e-215213553380,2018-02-01T00:00:00.000Z,2020-02-08T00:00:00.000Z,2020-02-15T05:42:41.161Z,2020-02-20T09:50:39.238Z
3,cab851b4-6419-4ebb-a755-7fca3ecea697,"Building construction Seed state, no UDM, AN-3458",Diverse geographies around the world,ec61ef60-d25b-407f-b678-f1a623893acd,2018-01-01T00:00:00.000Z,2018-02-01T00:00:00.000Z,2020-02-15T02:43:39.687Z,2020-02-20T09:50:39.218Z
4,f242fba0-0c64-497e-a871-0c1d182cfff3,"Douglas, Wyoming",Monthly Road Detection,36d9d1b0-dcd4-40bc-b411-5eb8e06062a2,2018-09-01T00:00:00.000Z,2019-09-01T00:00:00.000Z,2020-02-07T23:56:13.288Z,2020-02-20T09:41:18.141Z
5,3d560f9b-30c9-4fa7-ab91-6c88be2c49dc,Naples Oil Terminal,Daily Vessel Detection,1b4c9ee2-699b-4621-bb4f-934eda8090b5,2020-01-01T00:00:00.000Z,2020-03-31T00:00:00.000Z,2020-02-07T01:20:37.261Z,2020-02-20T09:51:29.093Z
6,604ae27b-e8d9-4db9-aef9-46551a58dc02,Cabello Container Terminal,Daily Vessel Detection,1b4c9ee2-699b-4621-bb4f-934eda8090b5,2020-01-01T00:00:00.000Z,2020-03-31T00:00:00.000Z,2020-02-07T01:19:36.562Z,2020-02-20T09:51:16.168Z
7,ba0c684a-1302-4f3b-8cb4-3946b56bc1fb,Jamaica_Buildings,COGINT 2020,b442c53b-fc72-4bee-bab4-0b7aa318ccd9,2019-11-01T00:00:00.000Z,2020-02-01T00:00:00.000Z,2020-02-06T16:09:38.094Z,2020-02-20T09:41:18.029Z
8,b3315105-88ee-4215-aa3e-3c2b050913ed,Jamaica_Roads,COGINT 2020,36d9d1b0-dcd4-40bc-b411-5eb8e06062a2,2019-11-01T00:00:00.000Z,2020-02-01T00:00:00.000Z,2020-02-06T16:08:42.087Z,2020-02-20T09:41:17.947Z
9,f75d5865-b3b9-4201-9a72-07a9ebebebae,Guatemala_5,COGINT 2020,36d9d1b0-dcd4-40bc-b411-5eb8e06062a2,2019-11-01T00:00:00.000Z,2020-02-01T00:00:00.000Z,2020-02-06T15:58:39.211Z,2020-02-20T09:41:21.006Z


#### Filtering subscriptions by feed

In [10]:
feed_id = feeds[0]['id']
feed_title = feeds[0]['title']
print(feed_title)
print('id:', feed_id)

Building construction, Global Monthly, no UDM: Change v1.3
id: 899de0a6-55de-4c2d-8f4e-215213553380


In [11]:
filtered_subscriptions_url = f'{BASE_URL}subscriptions?feedID={feed_id}'
print('url:', filtered_subscriptions_url)
resp = requests.get(filtered_subscriptions_url, auth=BASIC_AUTH)
filtered_subs = resp.json()['data']
filtered_sub_count = len(filtered_subs)
print(f'You have access to {filtered_sub_count} subscriptions for feed {feed_id} ({feed_title})')

url: https://api.planet.com/analytics/subscriptions?feedID=899de0a6-55de-4c2d-8f4e-215213553380
You have access to 2 subscriptions for feed 899de0a6-55de-4c2d-8f4e-215213553380 (Building construction, Global Monthly, no UDM: Change v1.3)


#### Inspecting a subscription's geometry
Subscriptions have a spatial area of interest described by a geojson geometry. We can visualize the area of interest for a subscription on a map.

In [12]:
# get the latest subscription's geometry
subscription = subs[0]
geom = subscription['geometry']
pprint(geom)

{'coordinates': [[[[46.2865483435179, 51.5238803899017],
                   [46.1424372793634, 51.5253724085522],
                   [46.1401931105756, 51.435472685135],
                   [46.2840212665101, 51.4339854325255],
                   [46.2865483435179, 51.5238803899017]]],
                 [[[31.3083874363058, 30.1230990949877],
                   [31.3083874363058, 29.7036857092783],
                   [31.8024105729672, 29.6972905877681],
                   [31.7992130122121, 30.1284283629128],
                   [31.3083874363058, 30.1230990949877]]],
                 [[[37.2421471540336, 36.2689431949095],
                   [37.1308712569163, 36.2672554193685],
                   [37.1330145628049, 36.1771473790386],
                   [37.2441630262575, 36.1788296294855],
                   [37.2421471540336, 36.2689431949095]]]],
 'type': 'MultiPolygon'}


In [18]:
# make a map, and draw the subscription geometry
from ipyleaflet import Map, GeoJSON
print(geom['coordinates'][0][0][0])
lon, lat = geom['coordinates'][0][0][0]

m = Map(center=(lat, lon), zoom=8)
geo_json = GeoJSON(data=subscription['geometry'], style = {'color': 'blue', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
m.add_layer(geo_json)
m

[46.2865483435179, 51.5238803899017]


Map(center=[51.5238803899017, 46.2865483435179], controls=(ZoomControl(options=['position', 'zoom_in_text', 'z…